In [1]:
from bert_serving.client import BertClient
from scipy import spatial
from numpy import argmax
from opencc import OpenCC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.externals import joblib

C:\Users\game2\Anaconda3\envs\BERT\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
cc = OpenCC('t2s')#因為bert目前還是用簡體字做分析，先轉成簡體字
bc = BertClient()#have to run Bert server beforehand
def consine_sim(v1,v2):#餘弦相似性公式
    return 1-spatial.distance.cosine(v1,v2)

# classes = ['房內服務行為','房內看菜單行為','房內點餐行為','服務項目',
#           '訂房行為','詢問健身房地點','詢問健身房開放時間','詢問會議中心地點',
#           '詢問會議中心開放時間','詢問游泳池地點','詢問游泳池開放時間','詢問禮品店地點',
#           '詢問禮品店開放時間','詢問餐廳地點','詢問餐廳開放時間']

classes = ['房內服務行為','詢問健身房地點','詢問餐廳開放時間']
sents_n_labels = [
    ('我要收垃圾，請找房務清潔人員',0),
    ('我要叫房內服務', 0),
    ('找人來幫我掃房間',0),
    ('垃圾太多請來收',0),
    ('房間太髒，需要人來清理',0),
    ('請幫我找管家來',0),
    ('我要換棉被，太薄了，請找管家來',0),
    ('我需要房務人員',0),
    ('請幫我找房務人員過來',0),
    ('房間好髒，需要清掃',0),
    ("健身房的位置在哪裡",1),
    ("你們有健身房嗎",1),
    ("我要去健身房",1),
    ("哪裡可以健身",1),
    ("健身房怎麼去",1),
    ("給我看健身房的位置",1),
    ("健身房在哪裡",1),
    ("健身房在幾樓",1),
    ("請問餐廳什麼時候營業",2),
    ("餐廳營業開放時間是幾點",2),
    ("餐廳開放中嗎",2),
    ("我現在要去餐廳用餐可以嗎",2),
    ("餐廳什麼時候開始",2),
    ("餐廳的開放時間",2),
    ("餐廳現在有開嗎",2),
    ("餐廳的營業時間",2)]


In [3]:
sents =[cc.convert(sent[0]) for sent in sents_n_labels]
print(sents)
labels = [label[1] for label in sents_n_labels]
print(labels)
classes = [cls for cls in classes]
print(classes)
classes_enc = bc.encode(classes)#把句子們轉換成數值向量

['我要收垃圾，请找房务清洁人员', '我要叫房内服务', '找人来帮我扫房间', '垃圾太多请来收', '房间太脏，需要人来清理', '请帮我找管家来', '我要换棉被，太薄了，请找管家来', '我需要房务人员', '请帮我找房务人员过来', '房间好脏，需要清扫', '健身房的位置在哪里', '你们有健身房吗', '我要去健身房', '哪里可以健身', '健身房怎么去', '给我看健身房的位置', '健身房在哪里', '健身房在几楼', '请问餐厅什么时候营业', '餐厅营业开放时间是几点', '餐厅开放中吗', '我现在要去餐厅用餐可以吗', '餐厅什么时候开始', '餐厅的开放时间', '餐厅现在有开吗', '餐厅的营业时间']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]
['房內服務行為', '詢問健身房地點', '詢問餐廳開放時間']


In [4]:
cc = OpenCC('t2s')# 繁轉簡
bc = BertClient()# 取得bert服務器資源
def cosine_sim(v1,v2):
    return 1-spatial.distance.cosine(v1,v2)
    


sents = [cc.convert(t[0]) for t in sents_n_labels ]
labels = [t[1] for t in sents_n_labels ]
classes = [cc.convert(s) for s in classes ]

classes_enc = bc.encode(classes)#把句子們轉換成數值向量
sents_enc = bc.encode(sents)

sents_enc_train, sents_enc_test, labels_train, labels_test =  \
train_test_split(sents_enc, labels, test_size=0.33,random_state=42)
cls = SVC()
cls.fit(sents_enc_train, labels_train)
score =cls.score(sents_enc_test, labels_test)
print('Accuracy:',score)


Accuracy: 0.7777777777777778


In [5]:
#save model
joblib.dump(cls,'cls.pkl')

['cls.pkl']